In [2]:
%pip install firebase-admin
%pip install google-cloud-firestore

     ---------------------------------------- 0.0/134.2 kB ? eta -:--:--
     ----------- --------------------------- 41.0/134.2 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 134.2/134.2 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/358.9 kB ? eta -:--:--
     ---------------- --------------------- 153.6/358.9 kB 3.1 MB/s eta 0:00:01
     ------------------------------ ------- 286.7/358.9 kB 2.9 MB/s eta 0:00:01
     -------------------------------------- 358.9/358.9 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/74.7 kB ? eta -:--:--
     ---------------------------------------- 74.7/74.7 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
     -- ------------------------------------- 0.3/4.3 MB 9.2 MB/s eta 0:00:01
     ---- ----------------------------------- 0.5/4.3 MB 5.6 MB/s eta 0:00:01
     ------- -------------------------------- 0.8/4.3 MB 6.0 MB/s eta 0:00:


[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import firebase_admin
from firebase_admin import credentials, firestore

# Bước 1: Load file JSON
cred = credentials.Certificate("../firestore_access_key.json")

# Bước 2: Khởi tạo app
firebase_admin.initialize_app(cred)

# Bước 3: Kết nối Firestore
db = firestore.client()

# Test: Lấy tất cả document trong collection "category"
docs = db.collection('category').stream()
for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

Commercial => {}
Fraud => {}
Others => {}
Work => {}


In [3]:
import random

for i in range(1, 11):
    doc_ref = db.collection('user').document('nmthuan21@clc.fitus.edu.vn').collection('mail').document(f'mail{i}')
    random_category = random.choice(['Work', 'Commercial', 'Fraud', 'Others'])
    doc_ref.set({
        'sender': 'hihi@gmail.com',
        'subject': f'This is mail number {i}',
        'categoryID': random_category,
        'description': f'This is a mail from {random_category} number {i}',
        'body': 'This is the body of the mail, which contains the content of the email. It can be very long and detailed. It can also include HTML content, images, and other media.',
})

In [9]:
import logging
from typing import List

# Initialize Firestore
def init_firestore():
    if not firebase_admin._apps:
        cred = credentials.Certificate("../firestore_access_key.json")
        firebase_admin.initialize_app(cred)
    return firestore.client()

# Add list of mails vào collection 'email'
def add_email_to_user(user_id: str, email_id: str, email_data: dict) -> bool:
    db = init_firestore()
    try:
        # Đường dẫn tới mail collection của user
        mail_ref = db.collection('user').document(user_id).collection('mail').document(email_id)
        
        # Set dữ liệu
        mail_ref.set(email_data)

        logging.info(f"Successfully added email {email_id} to user {user_id}.")
        return True
    
    except Exception as e:
        logging.error(f"Error adding email to user: {e}")
        return False


def query_emails_by_category(user_id: str, category_id: str) -> List[dict]:
    db = init_firestore()
    output = []

    try:
        mails = db.collection('user').document(user_id).collection('mail') \
            .where('categoryID', '==', category_id) \
            .stream()

        for mail_doc in mails:
            data = mail_doc.to_dict()
            data['id'] = mail_doc.id
            output.append(data)

        logging.info(f"Successfully fetched emails with category {category_id} for user {user_id}.")
    except Exception as e:
        logging.error(f"Error querying emails by category: {e}")

    return output

In [ ]:
add_email_to_user('nmthuan21@clc.fitus.edu.vn', 'mail1', {
    'categoryID': 'Work',
    'description': 'This is a work email'
})

In [20]:
email_from_work = query_emails_by_category('nmthuan21@clc.fitus.edu.vn', 'Fraud')

for email in email_from_work:
    print(f"ID: {email['id']}, Description: {email['description']}, Category: {email['categoryID']}")

ID: mail3, Description: This is a mail from Fraud number 3, Category: Fraud
ID: mail6, Description: This is a mail from Fraud number 6, Category: Fraud
ID: mail8, Description: This is a mail from Fraud number 8, Category: Fraud
